In [1]:
import numpy as np
import pandas as pd

In [56]:
df= pd.read_csv('./textfiles/HPRICE2.raw',delim_whitespace=True, header=None,  names=['price', 'crime', 'nox', 'rooms', 'dist', 'radial', 'proptax', 'stratio', 'lowstat', 'lprice', 'lnox', 'lproptax'])

#### 1. State the fundamental hypothesis under which the Ordinary Least Squares (OLS) estimators are unbiased. 

* The errors in the linear regression are uncorolated : $ Cov(\epsilon_i,\epsilon_j) = 0, \forall i \neq j $
* They have the same variance (homoscedastic) : $ Var(\epsilon_i) = \sigma^2 < \infty, \forall i $
* They all have a mean of zero : $ E[\epsilon_i] = 0, \forall i  $ 



#### 2. Show that under this assumption the OLS estimators are indeed unbiased.

$ y = X\beta + \epsilon $ and we estimate $\beta$ with $\hat\beta = (X^TX)^{-1}X^Ty$


$$ 

\begin{equation}
    \begin{aligned}
        E[\hat\beta] & = E[(X^TX)^{-1}X^Ty] \\
        & = E[(X^TX)^{-1}X^T(X\beta+\epsilon)] \\
        & = E[(X^TX)^{-1}X^TX\beta] + E[(X^TX)^{-1}X^T\epsilon) & \text{linearity of E} \\
        & = \beta + XE[\epsilon] \\
        &=  \beta & \text{as $E[\epsilon] = 0$ }

    \end{aligned}

\end{equation}

$$


#### 3. Explain the sample selection bias with an example from the course. 

The selection bias refers to a specific bias where your data (population) is not representative of the real data. The example given in class is about planes returning from war zones. People realised that plane returning from war zones had really specific area where they had been shot during their mission. They then upprgraded those planes to be more protected at these specifics areas to ensure less plane crash in combat. What they did not realised was that the plane comming home were not representative of the whole population wich implied a selection bias and therefore a bad conclusion. Indeed, the planes that crashed were not part of the selection for obvious reason. A more thourough reasoning would have been to protect the area where the plane that did come back were not shot as the one where they were shot did not affect their returning

#### 4. Explain the omitted variable bias with an example from the course

Ommited variable bias occurs when a variable wich has impact on the response variable (the Y variable), and is corrolated to the other variables in our statistical analysis is not taken into account. The example in class we talked about was regarding crops. If our mean is to improve our yield and we only look at fertilizer consumption data. We are missing the fact that lands have different quality for production. The land quality definitely have an impact on yield (wich is our mean) and it might also have an impact on fertilizer use. Farmer might prefer to put fertilizer on better land quality as it might improve its yield.

#### 5. Explain the problem of multicollinearity. Is it a problem in this dataset? 

If more than 2 columns of X are (nearly) linearly dependent , then $det(X^TX) = 0$ (or close to it) and it won't have an invert. In our case it means that it is gonna be numerically hard to compute the invert from $X^TX$

In more simple terms multicollinearity happens when two variables measure the exact same thing. (We can determine a variable directly based on another one or close to it)



In [57]:
df.head()


,price,crime,nox,rooms,dist,radial,proptax,stratio,lowstat,lprice,lnox,lproptax
0,24000,0.006,5.38,6.57,4.09,1,29.6,15.3,4.98,10.085810,1.682688,5.690360
1,21599,0.027,4.69,6.42,4.97,2,24.2,17.8,9.14,9.980402,1.545433,5.488938
2,34700,0.027,4.69,7.18,4.97,2,24.2,17.8,4.03,10.454500,1.545433,5.488938
3,33400,0.032,4.58,7.00,6.06,3,22.2,18.7,2.94,10.416310,1.521699,5.402678
4,36199,0.069,4.58,7.15,6.06,3,22.2,18.7,5.33,10.496790,1.521699,5.402678


In [64]:

y = df["price"]
X = df.iloc[:, 1:9]
log = df.iloc[:, 9:]

In [66]:

beta =  np.linalg.inv(X.T @ X)@X.T@y
np.corrcoef(X["crime"], X["proptax"]) 

array([[1.        , 0.58281922],
       [0.58281922, 1.        ]])

In [55]:
X

,1,2,3,4,5,6,7,8
0,0.006,5.38,6.57,4.09,1,29.6,15.3,4.98
1,0.027,4.69,6.42,4.97,2,24.2,17.8,9.14
2,0.027,4.69,7.18,4.97,2,24.2,17.8,4.03
3,0.032,4.58,7.00,6.06,3,22.2,18.7,2.94
4,0.069,4.58,7.15,6.06,3,22.2,18.7,5.33
...,...,...,...,...,...,...,...,...
501,0.063,5.73,6.59,2.48,1,27.3,21.0,9.67
502,0.045,5.73,6.12,2.29,1,27.3,21.0,9.08
503,0.061,5.73,6.98,2.17,1,27.3,21.0,5.64
504,0.110,5.73,6.79,2.39,1,27.3,21.0,6.48
